In [4]:
import numpy as np
import pandas as pd
import gurobipy as grb
from gurobipy import *
import csv
# # Read the dataset from a CSV file
# filename = "data_large_3dep.txt"
# data = pd.read_csv(filename, sep="\t")


with open(filename) as f:
    data = csv.reader(f, delimiter="\t")
    dataset = list(data)

for i in range(0,len(dataset)):
    for j in range(0,7):
        a = int(dataset[i][j])
        dataset[i][j] = a

# Extract the relevant data
# node_id = data["Node ID"].tolist()
# x_coord = data["X"].tolist()
# y_coord = data["Y"].tolist()
# a = data["Demand"].tolist()
# RT = data["Ready Time"].tolist()
# DT = data["Due Time"].tolist()
# ST = data["Service Time"].tolist()

dataset = np.array(dataset)
node_id = dataset[:, 0]  # Stores the id's of the nodes
x_coord = dataset[:, 1]  # Stores the x coordinates of all the nodes
y_coord = dataset[:, 2]  # Stores the y coordinates of all the nodes
a = dataset[:, 3]  # Stores the demand for each node
RT = dataset[:, 4]  # Stores the ready time for each node (earliest time for delivery)
DT = dataset[:, 5]  # Stores the due time for each node (latest time for delivery)
ST = dataset[:, 6]  # Stores the service time for each node (time needed for a delivery at each location )


# Define the capacities for each vehicle
b = np.array([300, 300, 300, 300, 300, 300])

# Initialize the Gurobi model
model = grb.Model("Vehicle Routing Problem")

# Define decision variables
x = model.addVars(len(node_id), len(node_id), len(b), vtype=grb.BINARY, name="If_Route_Travelled")
z = model.addVars(len(node_id), len(b), vtype=grb.BINARY, name="If_Node_Satisfied")
t = model.addVars(len(node_id), len(b), vtype=grb.CONTINUOUS, name="Service_Start_Time")
N = model.addVars(len(node_id), lb=0, ub=len(b), vtype=grb.INTEGER, name="Vehicle_Per_Depot")

# Define the objective function
obj = quicksum(x[i, j, v]*D[i, j] for i in node_id for j in node_id for v in b)
model.setObjective(obj, grb.MINIMIZE)

# Define constraints
# ... (Add the constraints here)
M = 2000

# # Ensures that every node is visited by only one vehicle
single_visit = model.addConstrs((quicksum(z[i, v] for v in V) == 1 for i in range(3, len(a))), 
                                "Single_Visit_Satisfaction")

# # Ensures that multiple vehicles can start from the Depot
depot_vehicle_num = model.addConstrs((quicksum(z[i, v] for v in V) == N[i] for i in range(3)), 
                                     "Vehicle_Leaving_Depot")
    
# # Total Number of Vehicles leaving the depots is K
total_vehicles = model.addConstr(quicksum(N[i] for i in range(3)) == K, "Total_Vehicles")    

# # Ensures that the sum of demand of a tour of each vehicle needs to be less than overall capacity of the vehicle
vehicle_cap = model.addConstrs((quicksum(a[i]*z[i, v] for i in I) <= b[v] for v in V), "Vehicle_Capacity")

# # Ensures that the number of vehicles leaving a node equals number of vehicles satisfying its demands
vehicle_departure = model.addConstrs((quicksum(x[i, j, v] for j in J) == z[i, v] for i in I for v in V), 
                                     "Vehicle_Departure_Constraint")

# # Ensures that each nodes is visited by particular vehicle v once
vehicle_arrival = model.addConstrs((quicksum(x[i, j, v] for i in I) == z[j, v] for j in J for v in V), 
                                   "Vehicle_Arrival_Constraint")

# # Defining the total time taken up to a certain node for a certain vehicle
service_start = model.addConstrs((quicksum(x[i, j, v]*(t[i, v] + ST[i] + D[i, j]) for i in I) <= t[j, v] 
                                  for j in range(3, len(a)) for v in V), 
                                 "Service_Start_Time_Constraint")

# # Ensures that there is no subtouring
subtour_constraint = model.addConstrs((t[i, v] + D[i, j] - M*(1 - x[i, j, v]) <= t[j, v] 
                                       for i in I for j in range(3, len(a)) for v in V), "Subtour_Constraint")

# # Ensures that the arrival of the vehicle is within the ready-time and due-time
ready_time_constraint = model.addConstrs((RT[i]*z[i, v] <= t[i, v] for i in I for v in V), "Ready_Time_Constraint")
due_time_constraint = model.addConstrs((t[i, v] <= DT[i]*z[i, v] for i in I for v in V), "Due_Time_Constraint")


# Set the Gurobi parameters
model.params.Heuristics = 0.5
model.params.PoolSolutions = 100
model.params.PoolSearchMode = 1
model.params.TimeLimit = 1800

# Solve the model
model.optimize()

# Print the solution
# ... (Print the solution here)

AttributeError: module 'gurobipy' has no attribute 'BINARY'